# 패키지 임포트

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from data_preprocessing import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer

# 데이터 로드

In [2]:
train_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-train-stemmed.csv')
valid_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-dev-stemmed.csv')
test_df = pd.read_csv('C:/Users/User/Desktop/r8/r8_dataset/r8-test-stemmed.csv')
train_df.head()

,text,edge,intent
0,champion product approv stock split champion p...,champion product approv stock split champion p...,earn
1,comput termin system cpml complet sale comput ...,comput termin system cpml complet sale comput ...,acq
2,cobanco inc cbco year net shr ct dlr net asset...,cobanco inc cbco year net shr ct dlr net asset...,earn
3,intern inc qtr jan oper shr loss two ct profit...,intern inc qtr jan oper shr loss two ct profit...,earn
4,brown forman inc bfd qtr net shr dlr ct net ml...,brown forman inc bfd qtr net shr dlr ct net ml...,earn


In [3]:
# train_df와 valid_df 결합
train_val_df = pd.concat([train_df, valid_df])

# 전처리

In [4]:
train_val_df['text']=train_val_df['text'].apply(clean_text)
test_df['text']=test_df['text'].apply(clean_text)

In [5]:
x_train=list(train_val_df['text'])
y_train=list(train_val_df['intent'])
x_test=list(test_df['text'])
y_test=list(test_df['intent'])

In [6]:
y=y_train+y_test

In [7]:
encoder=LabelEncoder()
encoder.fit(y)
label=encoder.transform(y)

y_train=list(label[:5484])
y_test=list(label[5484:])

In [9]:
to_txt=x_train+x_test

# tensor space model 생성

In [11]:
#불용어 불러오기
with open('C:/Users/user/Desktop/english.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [12]:
#문서 분류에 도움되는 10,000개 단어 선별
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
word_view=np.array(vect.get_feature_names_out())
feature_lst10000=word_view[TorF]

C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


In [13]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())
    
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(5484):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

train_max_feature=max(count_lst)

#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(2189):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

test_max_feature=max(count_lst)

max_feature=max(train_max_feature,test_max_feature)

Train_data에서 가장 많은 feature를 가진 문서의 경우 feature 232  개를 가짐
Train_data에서 가장 적은 feature를 가진 문서의 경우 feature 2  개를 가짐
Test_data에서 가장 많은 feature를 가진 문서의 경우 feature 237  개를 가짐
Test_data에서 가장 적은 feature를 가진 문서의 경우 feature 3  개를 가짐


In [16]:
max_feature

237

In [20]:
#1-Channel TextCuboid 생성
textcuboid=[]
for i in range(5484):
    frame1=np.zeros((max_feature,256)) #(max_feature, 256)
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
            frame1[idx_cnt]=np.load('./elmo_embedding/train(r8_256)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid.append(frame1)
textcuboid=np.array(textcuboid)
np.save('./1-Channel textcuboid_r8(elmo).npy',textcuboid)

In [21]:
#1-Channel TextCuboid 생성
textcuboid_test=[]
for i in range(2189):
    frame1=np.zeros((max_feature,256)) #(max_feature, 256)
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
            frame1[idx_cnt]=np.load('./elmo_embedding/test(r8_256)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid_test.append(frame1)
textcuboid_test=np.array(textcuboid_test)
np.save('./1-Channel textcuboid_test_r8(elmo).npy',textcuboid_test)